In [1]:
# Magics

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%matplotlib inline

import os, sys, gc
from tqdm import tqdm, tqdm_notebook, tqdm_pandas
from tqdm import trange
import time
from scipy import stats
from scipy.stats import shapiro

import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

from Bio import SeqIO
from collections import Counter

from multiprocessing import Pool, Process

import itertools
from modules.aa_properties import score_hydrophobicity_sequence, score_positions, score_sequence

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import ComplementNB

from joblib import dump, load

import warnings
warnings.filterwarnings('ignore')

In [3]:
aa =   ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
ignore = ["*", "U", "X"]
viruses = ['CMV_StrainAD169', 'YFV_Strain17D', 'HIV-1_StrainHXB2', 'HCV_StrainIsolateH', 'EBV_StrainAG876']
bacteria = ['SHGLsonnei', 'SLMLenteritidis', 'MYPLpneumoniae', 'MYBTsmegmatis', 'SLMLtyphimurium', 
            'YERSpseudotuberculosis', 'YERSenterocolitica', 'CLMDtrachomatis', 'MYPLsynoviae', 'KLEBpneumoniae',
            'CPBTjejuni', 'SHGLflexneri', 'CLOSdificile']
kidera = ["helix.bend.pref", "side.chain.size",
           "extended.str.pref", "hydrophobicity", "double.bend.pref", "partial.spec.vol",
           "flat.ext.pref", "occurrence.alpha.reg", "pK.C", "surrounding.hydrop"]
fts = ['ft1', 'ft2', 'ft3', 'ft4', 'ft5']

In [ ]:
fdict = {}
for key in os.listdir("data/fasta/"):
    if key.startswith("UP0"):
        fdict[key] = "Human"
    else:
        fdict[key] = key.strip("strain.fasta").strip("_")
        
def aa_prot_count(args):
    path, species = args
    record_dict = SeqIO.index("data/fasta/" + path, "fasta")
    df = pd.DataFrame(columns=['species', 'protein']+aa)
    aa_dict  = Counter()
    for key in record_dict.keys():
        aa_dict = aa_dict + Counter(record_dict[key].seq)
        for amino in list(aa_dict):
            if amino in ignore:
                del aa_dict[amino]
        df1 = pd.DataFrame.from_dict(aa_dict, orient='index').T
        df1['species'] = species
        df1['protein'] = key
        df1['prot_len'] = sum(aa_dict.values())
        df = pd.concat([df, df1])
    return df
    
def aa_kidera(args):
    path, species = args
    record_dict = SeqIO.index("data/fasta/" + path, "fasta")
    df = pd.DataFrame(columns=['species', 'protein']+kidera)
    for key in record_dict.keys():
        d = score_sequence(record_dict[key].seq, norm=True)
        df1 = pd.DataFrame(d).T
        df1.columns = kidera
        df1['species'] = species
        df1['protein'] = key
        df = pd.concat([df, df1])
    return df

def aa_hydrophobicity(args):
    path, species = args
    record_dict = SeqIO.index("data/fasta/" + path, "fasta")
    df = pd.DataFrame(columns=['species', 'protein', 'hydrophobicity'])
    for key in record_dict.keys():
        df1 = pd.DataFrame.from_dict({
                'hydrophobicity': score_hydrophobicity_sequence(record_dict[key].seq, norm=True),
                'species': species,
                'protein': key
            }, orient='index').T
        df = pd.concat([df, df1])
    return df

In [ ]:
with Pool(processes=len(fdict)) as pool:
    result = pool.map(aa_prot_count, fdict.items())

pdf = pd.concat(result, axis=0)
pdf = pdf.fillna(0)
pdf = pdf.reset_index(drop=True)

In [ ]:
pdf.head()

In [ ]:
pdf.shape, pdf.drop_duplicates(subset=aa).shape

In [ ]:
def count_aa_freq(row):
    return row[aa] / row['prot_len']
df = pdf.apply(lambda row: count_aa_freq(row), axis=1)

In [ ]:
df[['protein', 'species']] = pdf[['protein', 'species']]

In [ ]:
def group_name(name):
    if name in viruses:
        return 'virus'
    elif name in bacteria:
        return 'bacterium'
    elif name in 'Human':
        return 'human'

def cast_pca(df, n_comp, subset=aa):
    pca = PCA(n_components=n_comp)

    pctd = pca.fit_transform(df[subset])
    pcdf = pd.DataFrame(data = pctd, columns = ['pc' + str(i) for i in range(1,n_comp+1)])
    pcdf['species'] = df.species

    pcdf.species = pcdf.species.apply(lambda x: group_name(x))
    return pcdf, pca

In [ ]:
pcdf, pca = cast_pca(df, 10)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
evr = pca.explained_variance_ratio_
plt.plot([i for i in range(1, len(evr)+1)], np.cumsum(evr), 'ro')
# padding = 1 + 0.1 *  len(evr)
# ax.set_xlim(1 - padding, len(evr) + padding)
ax.set_title('Variance explained by new factors', fontsize=16)
ax.set_xlabel('Factors', fontsize=16)
ax.set_ylabel('Variance explained', fontsize=16)
ax.set_ylim(0, 1.05);

In [ ]:
pd.DataFrame(evr, columns=['ExplVar']).plot.bar(figsize=(8,6));

In [ ]:
NewAAFactors = pca.components_[:5]
NewAAFactors.shape

In [ ]:
myfactordf = pd.DataFrame(NewAAFactors.T, columns=fts, index=aa)
myfactordf.to_csv("modules/NewAAFactors.csv")

In [ ]:
pcdf.columns

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(8,8))
sns.scatterplot(ax=ax, x='pc2', y='pc1', hue='species', data=pcdf, s=50);

In [ ]:
def plot_pca(pcdf):
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('PC1', fontsize = 15)
    ax.set_ylabel('PC2', fontsize = 15)
    ax.set_title('PCA', fontsize = 20)

    targets = pcdf.species.unique()
    colours = ['r', 'g', 'b']
    for target, colour in zip(targets,colours):
        indicesToKeep = pcdf['species'] == target
        ax.scatter(pcdf.loc[indicesToKeep, 'pc1']
                   , pcdf.loc[indicesToKeep, 'pc2']
                   , c = colour
                   , s = 50)
    ax.legend(targets)
    ax.grid()
    
plot_pca(pcdf)

## Train Classifier

In [ ]:
fdf = df[~df['species'].isin(['SLMLenteritidis', 'EBV_StrainAG876'])]
fdf.shape

In [ ]:
fdf.species = fdf.species.apply(lambda x: group_name(x))
di = {'virus': 0, 'bacterium': 0, 'human': 1}
fdf.replace({'species': di}, inplace=True)
fdf[['species', 'protein']].head()

In [ ]:
clf = SVC(probability=True)

X_train, X_test, y_train, y_test = train_test_split(pca.transform(fdf[aa])[:,0:5], 
                                                    fdf['species'].values, 
                                                    test_size=.25)

scores = cross_val_score(clf, X=X_train, y=y_train, cv=5)
np.mean(scores)

In [ ]:
clf.fit(X=X_train, y=y_train)

In [ ]:
# dump(clf, "modules/hum_clf.jl")

In [ ]:
test_df = df[df['species'].isin(['SLMLenteritidis', 'EBV_StrainAG876', 'Human'])]
test_df['species'] = test_df.species.apply(lambda x: group_name(x))
test_df.replace({'species': di}, inplace=True)
X_ultratest = pca.transform(test_df[aa])[:,0:5]
y_ultratest = test_df.species

In [ ]:
print(classification_report(y_test, clf.predict(X_test), labels=[0, 1]))

In [ ]:
print(classification_report(y_ultratest, clf.predict(X_ultratest), labels=[0, 1]))

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(pca.transform(fdf[aa])[:,0:5], 
                                                    fdf['species'].values, 
                                                    test_size=.25)

scores = cross_val_score(clf, X=X_train, y=y_train, cv=5)
np.mean(scores)

In [ ]:
clf.fit(X=X_train, y=y_train)

In [ ]:
dump(clf, "modules/hum_clf.jl")

In [ ]:
print(classification_report(y_test, clf.predict(X_test), labels=[0, 1]))

In [ ]:
print(classification_report(y_ultratest, clf.predict(X_ultratest[:,0:5]), labels=[0, 1]))

## New Factors and Kidera factors correlation

In [ ]:
kid = pd.read_csv('modules/kidera.csv', header=None, names=kidera)
from modules.aa_properties import symbol_lookup
kid.index = kid.index.map(lambda x: symbol_lookup[x])
# kid

In [ ]:
factors.columns = fts
fact_df = pd.concat([kid, factors], axis=1)

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(fact_df.corr().loc[fts, kidera].T, annot=True, fmt='.2g')
sns.despine(ax=ax);

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(16,6))
sns.heatmap(factors.T, annot=True, fmt='.2g')
sns.despine(ax=ax);

## Let's take a look at immunogenic peptides

In [ ]:
cdf = pd.read_csv('data/chowell.csv')
cdf = cdf[['peptide', 'Immunogenicity']]
cdf.head()

In [ ]:
cdf.Immunogenicity = cdf.Immunogenicity.map({'Positive':1,'Negative':0})

In [ ]:
factors = pd.DataFrame(NewAAFactors, columns=aa).T
factors.shape

In [ ]:
def score_factors(sequence, norm=False):
    if norm:
        return factors.loc[list(sequence)].sum() / len(sequence)
    else:
        return factors.loc[list(sequence)].sum()
    
ndf = pd.concat([cdf, cdf.peptide.apply(lambda s: score_factors(s))], axis=1)
ndf.head()

In [ ]:
ndf.columns = ['peptide', 'immunogenicity'] + fts
fig, ax = plt.subplots(figsize=(8,6))
sns.scatterplot(x="ft1", y="ft2", hue='immunogenicity', data=ndf);

In [ ]:
ndf.columns = ['peptide', 'immunogenicity'] + fts

X_train, X_test, y_train, y_test = train_test_split(ndf[fts].values, ndf.immunogenicity, test_size=.25)
clf = RandomForestClassifier(n_estimators=400)

scores = cross_val_score(clf, X=X_train, y=y_train, cv=5)
np.mean(scores)

In [ ]:
scores

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 5, verbose=0, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train, y_train);

In [ ]:
rf_random.best_score_

## New factors and Kidera facors combined

In [ ]:
adf = pd.concat([
        cdf,
        cdf.peptide.apply(lambda s: score_factors(s)),
        cdf.peptide.apply(lambda s: score_sequence(s))
    ], axis=1)

adf.columns = ['peptide', 'Immunogenicity'] + fts + kidera
adf.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(adf[fts+kidera].values, adf.Immunogenicity, test_size=.25)
clf = RandomForestClassifier(n_estimators=400)

scores = cross_val_score(clf, X=X_train, y=y_train, cv=5)
np.mean(scores)

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 5, verbose=0, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train, y_train);

In [ ]:
rf_random.best_score_

## Kidera alone

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(adf[kidera].values, adf.Immunogenicity, test_size=.25)
clf = RandomForestClassifier(n_estimators=400)

scores = cross_val_score(clf, X=X_train, y=y_train, cv=5)
np.mean(scores)

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 5, verbose=0, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train, y_train);

In [ ]:
rf_random.best_score_

## With SVM self score

In [ ]:
def aa_new_factors(args):
    path, species = args
    record_dict = SeqIO.index("data/fasta/" + path, "fasta")
    df = pd.DataFrame(columns=['species', 'peptide'])
    for key in record_dict.keys():
        seq = record_dict[key]
        d = pd.Series([seq[i:i+9] for i in range(len(seq)-8)])
        df1 = pd.DataFrame(d, columns=['peptide'])
        df1['species'] = species
        df = pd.concat([df, df1])
    fdf = pd.concat([
            df,
            df.peptide.apply(lambda s: score_factors(s))
        ], axis=1)
    return fdf

In [ ]:
with Pool(processes=len(fdict)) as pool:
    result = pool.map(aa_new_factors, fdict.items())

fdf = pd.concat(result, axis=0)
fdf = fdf.reset_index(drop=True)

In [ ]:
fdf.head()